In [1]:
import torch
import cv2
import numpy as np
import ipywidgets.widgets as widgets
from IPython.display import display
from jetbot import Robot, Camera, bgr8_to_jpeg
import traitlets
import pathlib
import sys

pathlib.WindowsPath = pathlib.PosixPath
if 'pathlib._local' not in sys.modules:
    sys.modules['pathlib._local'] = pathlib
    
# ==========================================
# 1. YOLOv5 모델 불러오기
# ==========================================
print("모델을 로딩 중입니다... (시간이 조금 걸릴 수 있습니다)")
# 로컬에 있는 best.pt를 로드 (Jetson Nano의 GPU 사용)
model = torch.hub.load('ultralytics/yolov5:v7.0', 'custom', path='best.pt', force_reload=True)

# Jetson Nano는 GPU 메모리가 작으므로 반정밀도(FP16)를 쓰면 속도가 빨라질 수 있습니다 (선택사항)
# model.half() 
model.to('cuda') 
print("모델 로딩 완료!")

# ==========================================
# 2. 로봇 및 카메라 설정
# ==========================================
robot = Robot()
camera = Camera.instance(width=300, height=300) # YOLO 입력 크기에 맞춤 (300~640 권장)

# 결과를 보여줄 위젯 생성
image_widget = widgets.Image(format='jpeg', width=300, height=300)
label_widget = widgets.Label(value="대기 중...")

# 화면에 위젯 배치
display(widgets.VBox([
    widgets.HBox([image_widget, label_widget])
]))

# ==========================================
# 3. 실시간 추론 및 제어 함수 (여기가 핵심!)
# ==========================================
def execute(change):
    image = change['new']  # 카메라에서 새로운 이미지 받기 (numpy array)
    
    # YOLO 추론 실행
    results = model(image)
    
    # 결과 데이터프레임으로 변환
    df = results.pandas().xyxy[0]
    
    # --- 로봇 제어 로직 (이진 분류 활용) ---
    found_defect = False
    status_text = "정상 주행 (Normal)"
    
    if not df.empty:
        # 감지된 객체들 확인
        for index, row in df.iterrows():
            label = row['name']
            conf = row['confidence']
            
            # [조건] 만약 'Defect'(불량/장애물)가 감지되고 정확도가 0.5 이상이면
            if label == 'Defect' and conf > 0.5:
                found_defect = True
                break  # 하나라도 발견하면 즉시 반응
    
    if found_defect:
        robot.stop()  # 로봇 정지!
        status_text = "🚨 감지됨! (Defect) - 정지"
    else:
        # 여기서는 테스트를 위해 정지해두거나, 아주 느리게 전진시킬 수 있습니다.
        # robot.forward(0.1) 
        pass

    # --- 화면 업데이트 ---
    label_widget.value = status_text
    
    # 이미지에 바운딩 박스 그리기 (YOLOv5 내장 기능)
    # render()는 리스트를 반환하므로 첫 번째 요소([0])를 가져옵니다.
    annotated_frame = results.render()[0] 
    
    # 위젯에 이미지 송출
    image_widget.value = bgr8_to_jpeg(annotated_frame)

# ==========================================
# 4. 카메라 연결 (실행 시작)
# ==========================================
execute({'new': camera.value})  # 초기 1회 실행
camera.observe(execute, names='value')  # 카메라 값이 바뀔 때마다 execute 함수 실행

print("실시간 탐지가 시작되었습니다!")

모델을 로딩 중입니다... (시간이 조금 걸릴 수 있습니다)


Downloading: "https://github.com/ultralytics/yolov5/archive/v7.0.zip" to /root/.cache/torch/hub/v7.0.zip
YOLOv5 🚀 2025-12-23 Python-3.6.9 torch-1.7.0 CUDA:0 (NVIDIA Tegra X1, 3964MiB)



WARNING ⚠️ Python 3.7.0 is required by YOLOv5, but Python 3.6.9 is currently installed


Fusing layers... 
YOLOv5s summary: 157 layers, 7015519 parameters, 0 gradients
Adding AutoShape... 


모델 로딩 완료!


실시간 탐지가 시작되었습니다!


In [ ]:
camera.stop()